In [1]:
# %%
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler, PCA, VectorAssembler
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import col

# %%
spark = SparkSession.builder \
    .appName("SECOP_Transformaciones") \
    .master("local[*]") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/30 00:18:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Cargar datos transformados del notebook anterior
df = spark.read.parquet("/opt/spark-data/processed/secop_features.parquet")
print(f"Registros: {df.count():,}")
print(f"Columnas: {len(df.columns)}")

Registros: 1,000
Columnas: 20


In [3]:
df.show()

+-----------------------+-----------+--------------------+--------------------+------------+--------------------+------------------+--------------------+---------------+----------------------+---------------------+----+----+----------------+--------------------+-------------------+----------------+--------------------+-------------------+--------------------+
|referencia_del_contrato|nit_entidad|      nombre_entidad|        departamento|      ciudad|    tipo_de_contrato|valor_del_contrato|      fecha_de_firma|estado_contrato|valor_del_contrato_num|fecha_de_firma_parsed|anio| mes|departamento_idx|tipo_de_contrato_idx|estado_contrato_idx|departamento_vec|tipo_de_contrato_vec|estado_contrato_vec|        features_raw|
+-----------------------+-----------+--------------------+--------------------+------------+--------------------+------------------+--------------------+---------------+----------------------+---------------------+----+----+----------------+--------------------+--------------

In [4]:
scaler = StandardScaler(inputCol="features_raw",
                       outputCol = "features_scaled",
                       withMean=False,
                       withStd=True)

In [5]:
print("Aplicando StandardScaler")
scaler_model = scaler.fit(df)
df_scaled=scaler_model.transform(df)

df_scaled.show()

Aplicando StandardScaler
+-----------------------+-----------+--------------------+--------------------+------------+--------------------+------------------+--------------------+---------------+----------------------+---------------------+----+----+----------------+--------------------+-------------------+----------------+--------------------+-------------------+--------------------+--------------------+
|referencia_del_contrato|nit_entidad|      nombre_entidad|        departamento|      ciudad|    tipo_de_contrato|valor_del_contrato|      fecha_de_firma|estado_contrato|valor_del_contrato_num|fecha_de_firma_parsed|anio| mes|departamento_idx|tipo_de_contrato_idx|estado_contrato_idx|departamento_vec|tipo_de_contrato_vec|estado_contrato_vec|        features_raw|     features_scaled|
+-----------------------+-----------+--------------------+--------------------+------------+--------------------+------------------+--------------------+---------------+----------------------+-----------------

In [6]:
df_scaled.select("features_scaled").first()[0]

SparseVector(54, {0: 0.0012, 1: 2.1145, 32: 2.8226, 50: 7.5178})

In [7]:
k=25

pca= PCA( k=k,
         inputCol="features_scaled",
         outputCol="features_pca")

In [8]:
print("Aplicando PCA")

pca_model = pca.fit(df_scaled)
df_pca = pca_model.transform(df_scaled)

df_pca.select("features_pca").first()[0]

Aplicando PCA


26/01/30 00:18:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


DenseVector([1.0551, 1.4962, 0.1533, 2.1719, -2.7771, 2.2736, -1.3492, -0.0368, 0.7913, -1.1381, 0.6815, -0.4323, 0.8747, -1.0002, 0.9875, 1.6206, -0.6204, -0.3367, 0.2656, 0.4688, -0.1257, -0.1813, 0.062, 0.1999, 0.0193])

In [9]:
explained_variance = pca_model.explainedVariance
sum(explained_variance)

0.5692633737728403

In [10]:
pipeline_transform = Pipeline(stages=[scaler, pca])
pipeline_transform_model=pipeline_transform.fit(df)
df_final=pipeline_transform_model.transform(df)

df_ml=df_final.select("features_pca","valor_del_contrato_num")



In [11]:
pipeline_path= "/opt/spark-data/processed/transformation_pipeline"
output_path= "/opt/spark-data/processed/secop_ml_ready.parquet"
df_ml.write.mode("overwrite").parquet(output_path)
pipeline_transform_model.save(pipeline_path)

26/01/30 00:18:36 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
26/01/30 00:18:37 WARN FileUtil: Failed to delete file or dir [/opt/spark-data/processed/transformation_pipeline/metadata/_temporary]: it still exists.
26/01/30 00:18:39 WARN FileUtil: Failed to delete file or dir [/opt/spark-data/processed/transformation_pipeline/stages/0_StandardScaler_dd6d85fac75e/metadata/_temporary]: it still exists.
26/01/30 00:18:40 WARN FileUtil: Failed to delete file or dir [/opt/spark-data/processed/transformation_pipeline/stages/0_StandardScaler_dd6d85fac75e/data/_temporary]: it still exists.
26/01/30 00:18:43 WARN FileUtil: Failed to delete file or dir [/opt/spark-data/processed/transformation_pipeline/stages/1_PCA_52c4b349aab5/data/_temporary]: it still exists.
